In [ ]:
man locate

In [ ]:
# reducer.sh
awk '{line_count += $1} END { print line_count}'

In [ ]:
HADOOP_STREAMING_JAR = '/path_to_jar'
yarn jar $HADOOP_STREAMING_JAR \
-mapper 'wc -l' \
-reducer './reducer.sh' \
-files reducer.sh \
-numReduceTasks 1 \
-input /data/wiki/articles \
-output wc_folder          

In [ ]:
#mapper.py
import sys

line_count = 0
for line in sys.stdin:
    line_count += 1
print(line_count)

In [ ]:
# reducer.py
import sys
line_count = sum(int(value) for value in sys.stdin)
print(line_count)

In [ ]:
HADOOP_STREAMING_JAR = '/path_to_jar'
yarn jar $HADOOP_STREAMING_JAR \
-files mapper.py, reducer.py \
-mapper 'python mapper.py' \
-reducer './reducer.sh' \
-numReduceTasks 1 \
-input /data/wiki/articles \
-output wc_folder          

# fully in python

In [2]:
#mapper.py

import re, sys
for line in sys.stdin:
    article_id, content = line.split("\t", 1)
    words = re.split("\W+", content)
    for word in words:
        if word:
            print(word, 1, sep="\t")
            

In [3]:
# reducer.py
import sys
current_word = None
word_count = 0
for line in sys.stdin:
    word, counts = line.split("\t", 1)
    counts = int(counts)
    if word == current_word:
        word_count += counts
    else:
        if current_word:
            print(current_word, word_count, sep="\t")
        current_word = word
        word_count = counts
if current_word:
    print(current_word, word_count, sep="\t")

In [ ]:
HADOOP_STREAMING_JAR = '/path_to_jar'
yarn jar $HADOOP_STREAMING_JAR \
-files mapper.py, reducer.py \
-mapper 'python mapper.py' \
-reducer 'python reducer.py' \
-numReduceTasks 1 \
-input /data/wiki/articles \
-output wc_folder    

In [ ]:
tar -cf names.tar male.txt female.txt

#mapper.py

import re, sys



def read_vocab(file_path):
    return set(word.strip() for word in open(file_path))

male_names = read_vocab("names.tar/male.txt")
female_names = read_vocab("names.tar/female.txt")

    
for line in sys.stdin:
    article_id, content = line.split("\t", 1)
    words = re.split("\W+", content)
    for word in words:
        if word:
            if word in male_names or word in female_names:
                print(word, 1, sep="\t")
#####            
            
            
HADOOP_STREAMING_JAR = '/path_to_jar'
yarn jar $HADOOP_STREAMING_JAR \
-files mapper.py, reducer.py \
-archives names.tar
-mapper 'python mapper.py' \
-reducer 'python reducer.py' \
-numReduceTasks 1 \
-input /data/wiki/articles \
-output wc_folder    

# Environment


In [ ]:
HADOOP_STREAMING_JAR = '/path_to_jar'

yarn jar $HADOOP_STREAMING_JAR -D word_pattern="\w+\d+" \
-files mapper.py, reducer.py \
-archives names.tar
-mapper 'python mapper.py' \
-reducer 'python reducer.py' \
-numReduceTasks 1 \
-input /data/wiki/articles \
-output wc_folder